# Get Data For Route Finder 2.0

In [ ]:
import pandas as pd
import geopandas as geo_pd
import json

''' LOCAL IMPORTS'''

from getPop_df import makeAPI_PopByBlock2010
from myAPIs import census_API_key as key
from downloadGEOdata import Download_Unzip_GEOs
from joinBlockPopGEOs import joinDFs


## Get Population Data by Census City Block 

In [ ]:
key = key()
state = str(input('enter state FIPS code (i.e. 48 for TX): '))
county = str(input('enter county FIPS code (i.e. 453 for Travis): '))

In [ ]:
df = makeAPI_PopByBlock2010(state, county, key)

In [ ]:
df.head()

## Download Block-level shapefiles for State and Join DataFrames

The downloaded data includes census block shapes for the entire state specified. This may take a while to download, unzip, and transform. 

In [ ]:
# BlockGeoPath = Download_Unzip_GEOs(state)

In [ ]:
df = joinDFs(BlockGeoPath, df)